In [4]:
import os
import pandas as pd
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
from tqdm.notebook import tqdm
import logging
from typing import List

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# InfluxDB configuration
bucket = "FinancialData"
org = "Datapine"
token = "hYMwYdwT5-Lbe8vP8oh40FDZ46LaTrWRapXB9X9tul2HiPG4IOKBoBGbLBUIRcjND9qh40RjQw1beG8iRpr7ZQ=="
url = "http://localhost:8086"

# Optimized client configuration
client = InfluxDBClient(
    url=url, 
    token=token, 
    org=org,
    timeout=30000,  # 30 second timeout
    retries=3
)

# Use synchronous write API with batching
write_api = client.write_api(write_options=SYNCHRONOUS)

column_names = [
    "open_time", "open", "high", "low", "close", "volume",
    "close_time", "quote_asset_volume", "number_of_trades",
    "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"
]

# Columns to convert to numeric (excluding time columns and ignore)
numeric_columns = [
    "open", "high", "low", "close", "volume", "quote_asset_volume",
    "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume"
]

def create_points_batch(df: pd.DataFrame, symbol: str) -> List[Point]:
    """Create a batch of InfluxDB points from DataFrame rows."""
    points = []
    
    for row in df.itertuples(index=False):
        try:
            point = (
                Point("klines")
                .tag("symbol", symbol)
                .field("open", row.open)
                .field("high", row.high)
                .field("low", row.low)
                .field("close", row.close)
                .field("volume", row.volume)
                .field("quote_asset_volume", row.quote_asset_volume)
                .field("number_of_trades", int(row.number_of_trades))
                .field("taker_buy_base_asset_volume", row.taker_buy_base_asset_volume)
                .field("taker_buy_quote_asset_volume", row.taker_buy_quote_asset_volume)
                .time(int(row.open_time), WritePrecision.MS)
            )
            points.append(point)
        except (ValueError, TypeError) as e:
            logger.warning(f"Skipping row due to data error: {e}")
            continue
    
    return points

def process_csv_file(fpath: str, symbol: str, batch_size: int = 5000) -> int:
    """Process a single CSV file and write to InfluxDB in batches."""
    try:
        # Read CSV with optimized settings
        df = pd.read_csv(
            fpath, 
            names=column_names, 
            header=None,
            dtype={
                'open_time': 'str',  # Read as string first for validation
                'number_of_trades': 'Int64'  # Handle potential NaN values
            }
        )
        
        # Filter valid timestamps more efficiently
        df = df[df["open_time"].str.isdigit()].copy()
        
        if df.empty:
            logger.warning(f"No valid data in {fpath}")
            return 0
        
        # Drop ignore column
        df = df.drop(columns=["ignore"])
        
        # Convert numeric columns efficiently
        for col in numeric_columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # Convert open_time to int64
        df['open_time'] = df['open_time'].astype('int64')
        
        # Drop rows with any NaN values
        initial_rows = len(df)
        df = df.dropna()
        if len(df) < initial_rows:
            logger.info(f"Dropped {initial_rows - len(df)} rows with invalid data in {fpath}")
        
        # Process in batches
        total_rows = len(df)
        rows_written = 0
        
        for i in range(0, total_rows, batch_size):
            batch_df = df.iloc[i:i + batch_size]
            points = create_points_batch(batch_df, symbol)
            
            if points:
                write_api.write(bucket=bucket, org=org, record=points)
                rows_written += len(points)
        
        logger.info(f"Successfully imported {rows_written} rows from {fpath}")
        return rows_written
        
    except Exception as e:
        logger.error(f"Error processing {fpath}: {e}")
        return 0

def main():
    """Main execution function."""
    data_dir = "binance_klines"
    total_files_processed = 0
    total_rows_imported = 0
    
    # Get list of symbol directories
    symbol_dirs = [d for d in os.listdir(data_dir) 
                   if os.path.isdir(os.path.join(data_dir, d))]
    
    try:
        for symbol in tqdm(symbol_dirs, desc="Processing symbols"):
            symbol_dir = os.path.join(data_dir, symbol)
            
            # Get CSV files in symbol directory
            csv_files = [f for f in os.listdir(symbol_dir) if f.endswith(".csv")]
            
            for fname in tqdm(csv_files, desc=f"Files in {symbol}", leave=False):
                fpath = os.path.join(symbol_dir, fname)
                rows_imported = process_csv_file(fpath, symbol)
                total_rows_imported += rows_imported
                total_files_processed += 1
        
        logger.info(f"Import completed! Processed {total_files_processed} files, "
                   f"imported {total_rows_imported} total rows")
        
    except KeyboardInterrupt:
        logger.info("Import interrupted by user")
    except Exception as e:
        logger.error(f"Unexpected error during import: {e}")
    finally:
        client.close()
        logger.info("InfluxDB client closed")

if __name__ == "__main__":
    main()

Processing symbols:   0%|          | 0/480 [00:00<?, ?it/s]

Files in AVAAIUSDT: 0it [00:00, ?it/s]

Files in FORMUSDT: 0it [00:00, ?it/s]

Files in ARCUSDT: 0it [00:00, ?it/s]

Files in AUSDT: 0it [00:00, ?it/s]

...

ERROR:__main__:Error processing binance_klines/DEGOUSDT/DEGOUSDT-1m-2024-12.csv: Unable to parse string "count" at position 0


Files in DEEPUSDT: 0it [00:00, ?it/s]

Files in NEWTUSDT: 0it [00:00, ?it/s]

Files in ZKUSDT:   0%|          | 0/7 [00:00<?, ?it/s]

ERROR:__main__:Error processing binance_klines/ZKUSDT/ZKUSDT-1m-2024-09.csv: Unable to parse string "count" at position 0
ERROR:__main__:Error processing binance_klines/ZKUSDT/ZKUSDT-1m-2024-08.csv: Unable to parse string "count" at position 0
ERROR:__main__:Error processing binance_klines/ZKUSDT/ZKUSDT-1m-2024-06.csv: Unable to parse string "count" at position 0
ERROR:__main__:Error processing binance_klines/ZKUSDT/ZKUSDT-1m-2024-12.csv: Unable to parse string "count" at position 0
ERROR:__main__:Error processing binance_klines/ZKUSDT/ZKUSDT-1m-2024-07.csv: Unable to parse string "count" at position 0
ERROR:__main__:Error processing binance_klines/ZKUSDT/ZKUSDT-1m-2024-11.csv: Unable to parse string "count" at position 0
ERROR:__main__:Error processing binance_klines/ZKUSDT/ZKUSDT-1m-2024-10.csv: Unable to parse string "count" at position 0
INFO:__main__:Import completed! Processed 8111 files, imported 35376922 total rows
INFO:__main__:InfluxDB client closed


In [5]:
import pandas as pd
from influxdb_client import InfluxDBClient

# InfluxDB configuration
bucket = "FinancialData"
org = "Datapine"
token = "hYMwYdwT5-Lbe8vP8oh40FDZ46LaTrWRapXB9X9tul2HiPG4IOKBoBGbLBUIRcjND9qh40RjQw1beG8iRpr7ZQ=="
url = "http://localhost:8086"

def get_btc_ohlcv(days_back=7):
    """
    Fetch OHLCV data for BTCUSDT from InfluxDB
    
    Args:
        days_back (int): Number of days to look back (default: 7)
    
    Returns:
        pandas.DataFrame: DataFrame with columns [time, open, high, low, close, volume]
    """
    
    client = InfluxDBClient(url=url, token=token, org=org)
    query_api = client.query_api()
    
    query = f'''
    from(bucket: "{bucket}")
    |> range(start: -{days_back}d)
    |> filter(fn: (r) => r._measurement == "klines")
    |> filter(fn: (r) => r.symbol == "BTCUSDT")
    |> filter(fn: (r) => r._field == "open" or r._field == "high" or r._field == "low" or r._field == "close" or r._field == "volume")
    |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
    |> sort(columns: ["_time"])
    |> keep(columns: ["_time", "open", "high", "low", "close", "volume"])
    '''
    
    try:
        result = query_api.query_data_frame(query)
        
        if not result.empty:
            # Rename _time to time for cleaner output
            result = result.rename(columns={'_time': 'time'})
            # Convert time to datetime if it isn't already
            result['time'] = pd.to_datetime(result['time'])
            # Reset index for cleaner display
            result = result.reset_index(drop=True)
            
        return result
        
    except Exception as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame()
    
    finally:
        client.close()

# Usage examples
if __name__ == "__main__":
    
    # Get last 7 days of BTCUSDT data
    btc_data = get_btc_ohlcv(days_back=7)
    
    if not btc_data.empty:
        print("BTCUSDT OHLCV Data:")
        print("=" * 50)
        print(f"Total records: {len(btc_data)}")
        print(f"Time range: {btc_data['time'].min()} to {btc_data['time'].max()}")
        print("\nFirst 5 rows:")
        print(btc_data.head())
        print("\nLast 5 rows:")
        print(btc_data.tail())
        
        # Basic statistics
        print("\nPrice Statistics:")
        print(f"Highest price: ${btc_data['high'].max():,.2f}")
        print(f"Lowest price: ${btc_data['low'].min():,.2f}")
        print(f"Latest close: ${btc_data['close'].iloc[-1]:,.2f}")
        print(f"Total volume: {btc_data['volume'].sum():,.2f}")
        
    else:
        print("No data found for BTCUSDT")

# Alternative: Get specific number of latest records
def get_btc_latest(limit=100):
    """Get the latest N records for BTCUSDT"""
    
    client = InfluxDBClient(url=url, token=token, org=org)
    query_api = client.query_api()
    
    query = f'''
    from(bucket: "{bucket}")
    |> range(start: -30d)
    |> filter(fn: (r) => r._measurement == "klines")
    |> filter(fn: (r) => r.symbol == "BTCUSDT")
    |> filter(fn: (r) => r._field == "open" or r._field == "high" or r._field == "low" or r._field == "close" or r._field == "volume")
    |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
    |> sort(columns: ["_time"], desc: true)
    |> limit(n: {limit})
    |> sort(columns: ["_time"])
    |> keep(columns: ["_time", "open", "high", "low", "close", "volume"])
    '''
    
    try:
        result = query_api.query_data_frame(query)
        if not result.empty:
            result = result.rename(columns={'_time': 'time'})
            result['time'] = pd.to_datetime(result['time'])
            result = result.reset_index(drop=True)
        return result
    except Exception as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame()
    finally:
        client.close()

# Quick one-liner usage:
# btc_data = get_btc_ohlcv(days_back=1)  # Last 24 hours
# btc_latest = get_btc_latest(limit=50)   # Latest 50 records

No data found for BTCUSDT
